In [2]:
import types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
client_3f024b86b4d943669e03a09b79c6cd24 = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='ktqZDqvOxNx-UYxRKihB2MrU8ptO5N412NcfE3vuIT4y',
    ibm_auth_endpoint="https://iam.au-syd.bluemix.net/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.ap-geo.objectstorage.service.networklayer.com')

body = client_3f024b86b4d943669e03a09b79c6cd24.get_object(Bucket='dataoftoronto-donotdelete-pr-pio7pxrqan1a8h',Key='PostalCode_Toronto.csv')['Body']
# add missing __iter__ method, so pandas accepts body as file-like object
if not hasattr(body, "__iter__"): body.__iter__ = types.MethodType( __iter__, body )

# If you are reading an Excel file into a pandas DataFrame, replace `read_csv` by `read_excel` in the next statement.
df = pd.read_csv(body)
df.head()

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


Delete any rows that contain "Not assigned"

In [19]:
df=df[df['Borough'].str.contains("Not assigned") == False]
df.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M6A,North York,Lawrence Heights
6,M6A,North York,Lawrence Manor


Using **.groupby** to group the neighbourhoods with the same postal code and join them together by **lambda**

In [8]:
df1 = df.groupby("Postcode", as_index = False).agg(lambda x:', '.join(set(x)))
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


Using **.loc** to find row where Neighbourhood is not assigned, and replace it with the same data at "Borough" column

In [20]:
df1.loc[df1['Neighbourhood']=='Not assigned', "Neighbourhood"]=df1.loc[df1['Neighbourhood']=="Not assigned", "Borough"]
df1.head()

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek"
2,M1E,Scarborough,"Guildwood, West Hill, Morningside"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


The last thing is print out the shape of dataframe by **.shape**

In [6]:
df1.shape

(103, 2)

Load the location coordinate data

In [18]:
geo = pd.read_csv(r'http://cocl.us/Geospatial_data')
geo.drop('Postal Code', axis = 1, inplace = True)
geo.head()

,Latitude,Longitude
0,43.806686,-79.194353
1,43.784535,-79.160497
2,43.763573,-79.188711
3,43.770992,-79.216917
4,43.773136,-79.239476


In [17]:
tc = df1.join(geo)
tc.head()

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Port Union, Rouge Hill, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, West Hill, Morningside",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
